# Ages of Athletes 
This is the notebook for getting the information on the ages of each individual athlete. As it stands there are 2680 athletes out of 25825 that we cannot get the age for. We have also not decided if we are going to actually use the age field yet so it might not be nessecary to get the remaining ages. 

In [2]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from bs4 import BeautifulSoup
import webbrowser
import urllib.request
from lxml import html
import zipfile
import re
import string
import sys, os

In [3]:
# Ensure the file exists
if not os.path.exists(r"..\..\data\prep\Games\Games-500.csv" ):
    print("Missing dataset file")

In [4]:
df = pd.read_csv(r"..\..\data\prep\Games\Games-500.csv", encoding = "ISO-8859-1")

# Function to scrape the Olympic Website
Each athlete that has played in the olympic games has there own individual page on the olympic website. On these pages are some perosnal details as well as games history. On every page is the athletees DOB this is what we want from each athlete when we scrape. The function below goes to the area of the html page that contains the DOB, it is the same in every page and scrapes and returns the year of birth. Each atheltes page is this url https://www.olympic.org/{} with the ahtletes name on the end the probelm is sometimes athletes have up to 5 names and the names used on the url are a mix of these not always consitent. 

In [10]:
# Function to get the age of an olympian from the olympics website 
def getAge(name):
    
    # The basic url without the name of athlete at the end 
    url = "https://www.olympic.org/{}".format(name)
    try:
        with urllib.request.urlopen(url) as response:
            page = response.read()
        soup = BeautifulSoup(page, 'html.parser')
        
        #cfinds all the text-box on the html page
        result = soup.find_all("div", {"class":"text-box"})
        #the DOB is always in the 4th one on the pages
        print(result[3].text)
        
        # the rest of the code breaks up the text box extracting only the DOB
        string = result[3].text
        string = string.replace("\r", "")
        string  = string.replace("\n", " ")
        if(string[1] == 'B'):
            string = string[14:18]
        elif(string[1] == 'L'):
            string = string[15:19]
        else:
            string = None
        return string
    except:
        print ("ERROR - "+url)

In [ ]:
# Simple for loop if the city is in the same country as the athlete is from set true to Home_Adv else false
for x in range(len(df)):
    index = df.iloc[x].name 
    year_of_games = df['Year'].iloc[x]
    name = df['Ath_Name'].iloc[x]
    
    print("Row number: ", index)
    namL = name.split(',')
    
    if(len(namL) > 1):
        name = name.lower()
        name = name.replace('ö', 'o')
        translator = str.maketrans('', '', string.punctuation)
        namL = name.split(',')

        firstname = namL[1].translate(translator)
        lastname = namL[0].translate(translator)

        comb_name = firstname + " " +  lastname
        comb_name = comb_name.split()

        fullname = '-'.join(comb_name)
        fullname = fullname.lower()

    try:
        age = int(getAge(fullname))
        df.set_value(index, 'Age', year_of_games - age)
    except: 
        continue

In [ ]:
df.to_csv(r"..\..\data\prep\Games\Games-550.csv", index=False)